In [ ]:
using BenchmarkTools
using Plotly
using Random
using StatsBase

In [ ]:
Random.seed!(20121020)

;

In [ ]:
function cumsum_reverse(a::Vector)

    return reverse(cumsum(Iterators.reverse(a)))

end

In [ ]:
element_ = string.(Array('A':'Z'))

n_element = length(element_)

element_score_ = randn(n_element)

;

In [ ]:
n_set_element = 3

set_element_ = sample(element_, n_set_element; replace = false);

In [ ]:
index_ = sortperm(element_score_)

element_score_ = element_score_[index_]

# element_ = element_[index_]

plot([scatter(y = element_score_)], Layout(title = "Element Score"))

## Is

In [ ]:
set_element_to_nothing = Dict(set_element => nothing for set_element in set_element_)

is_h = [Float64(haskey(set_element_to_nothing, element)) for element in element_]

is_m = 1 .- is_h

plot(
    [scatter(name = "Hit", y = is_h), scatter(name = "Miss", y = is_m)],
    Layout(title = "Is"),
)

In [ ]:
is_h_p = is_h / sum(is_h)

is_h_p_rc = cumsum(is_h_p)

is_h_p_lc = cumsum_reverse(is_h_p)

plot(
    [
        scatter(name = "Probability", y = is_h_p),
        scatter(name = "Cumulative probability >", y = is_h_p_rc),
        scatter(name = "Cumulative probability <", y = is_h_p_lc),
    ],
    Layout(title = "Is Hit"),
)

In [ ]:
is_m_p = is_m / sum(is_m)

is_m_p_rc = cumsum(is_m_p)

is_m_p_lc = cumsum_reverse(is_m_p)

plot(
    [
        scatter(name = "Probability", y = is_m_p),
        scatter(name = "Cumulative probability >", y = is_m_p_rc),
        scatter(name = "Cumulative probability <", y = is_m_p_lc),
    ],
    Layout(title = "Is Miss"),
)

## Amplitutde

In [ ]:
a = abs.(element_score_)

;

In [ ]:
a_p = a / sum(a)

a_p_rc = cumsum(a_p)

a_p_lc = cumsum_reverse(a_p)

plot(
    [
        scatter(name = "Probability", y = a_p),
        scatter(name = "Cumulative probability >", y = a_p_rc),
        scatter(name = "Cumulative probability <", y = a_p_lc),
    ],
    Layout(title = "Amplitude"),
)

In [ ]:
a_h = h_1 .* a

a_h_p = a_h / sum(a_h)

a_h_p_rc = cumsum(a_h_p)

a_h_p_lc = cumsum_reverse(a_h_p)

plot(
    [
        scatter(name = "Probability", y = a_h_p),
        scatter(name = "Cumulative probability >", y = a_h_p_rc),
        scatter(name = "Cumulative probability <", y = a_h_p_lc),
    ],
    Layout(title = "Amplitude Hit"),
)

In [ ]:
a_m = m_1 .* a

a_m_p = a_m / sum(a_m)

a_m_p_rc = cumsum(a_m_p)

a_m_p_lc = cumsum_reverse(a_m_p)

plot(
    [
        scatter(name = "Probability", y = a_m_p),
        scatter(name = "Cumulative probability >", y = a_m_p_rc),
        scatter(name = "Cumulative probability <", y = a_m_p_lc),
    ],
    Layout(title = "Amplitude Miss"),
)

## KS

In [ ]:
# KS

s = h_1_p_lc - m_1_p_lc

plot([scatter(y = s)])

## JSD

In [ ]:
# # JSD

# s = JSD(h_a_p_rc, m_a_p_rc, a_p_rc) - JSD(h_a_p_lc, m_a_p_lc, a_p_lc)

## Score

In [ ]:
score = sum(s) / length(s)